# Create chm_fp collection in genra_dev_v4

In [4]:
# %load ../env.py
%reload_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 

#Set environment variables


# Set up the local source files
TOP = '/'.join(os.getcwd().split('/')[:-2])+'/'
#TOP = "/share/home/ishah/ipynb/pb/HTTR_WF/"

LIB = TOP+'lib'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

os.environ['PYTHONPATH']=LIB


DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

if not os.path.exists(DAT_DIR): os.mkdir(DAT_DIR)
if not os.path.exists(FIG_DIR): os.mkdir(FIG_DIR)


from db.mongo import *
from db.fpsim import *
from chm.mkfp import *
DB = openMongo(db='genra_dev_v4')


Populating the interactive namespace from numpy and matplotlib


# Parallel FP generation
`ipcluster start --profile=pb_parallel --n=25`

In [11]:
import ipyparallel as PP

RC=PP.Client(profile='pb_parallel')
d_view=RC[:]
lb_view = RC.load_balanced_view()
lb_view.block = True
x=file("../parenv.py",'r').read()
d_view.execute(x)


<AsyncResult: execute>

In [6]:
from chm.mkfp import *

In [13]:
d_view.execute("from chm.mkfp import *")
#d_view.execute("DB = openMongo(db='genra_dev_v4')")

<AsyncResult: execute>

## Test threaded makeFP

In [ ]:
from threading import Thread
SID = []

def procCurs(curs,res):
    for C in curs:
        makeFP(C['dsstox_sid'], col_comp=DB.compounds,col_chm_fp=DB.chm_fp,
               save=True,replace=False)
        
        
Curs = DB.compounds.parallel_scan(50)
TH0  = [Thread(target=procCurs,args=(curs,SID)) for curs in Curs]
for th in TH0: th.start()
for th in TH0: th.join()


## How many chemicals left to do

In [ ]:
DONE = DB.chm_fp.distinct('dsstox_sid')
WORK = []

for C in DB.compounds.find():
    if DB.chm_fp.find(dict(dsstox_sid=C['dsstox_sid'])).count()>0: 
        continue
    else:
        WORK.append(C['dsstox_sid'])
#list(set(DB.compounds.distinct('dsstox_sid')).difference(DONE))
len(WORK)

In [18]:
WORK = []
for j in xrange(0,DB.compounds.count(),100000):
    WORK += [i['dsstox_sid'] for i in DB.compounds.find().skip(j).limit(100000)]
    if j % 100000==0: 
        print j

0
100000
200000
300000
400000
500000
600000
700000


In [19]:
len(WORK)

750464

## Run in parallel

In [20]:
import datetime
from chm.mkfp import *

            
print "Start ... ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",len(WORK)

lb_view.map(lambda dtxsid: makeFP(dtxsid, col_comp=DB.compounds,col_chm_fp=DB.chm_fp,save=True,replace=False),WORK)


print "End ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",len(WORK)



Start ...  2017-12-20 18:41:23 tasks =  750464
End  2017-12-21 08:14:33 tasks =  750464


In [21]:

for i in ['name','mrgn.ds','mrgn.n','httr.n','httr.ds','casrn',
          'dsstox_cid','dsstox_sid']:
    print i
    DB.chm_fp.create_index(i)


name
mrgn.ds
mrgn.n
httr.n
httr.ds
casrn
dsstox_cid
dsstox_sid
